In [0]:
import pandas as pd
import numpy as np
from keras.models import Sequential
# from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
# from keras.layers.normalization import BatchNormalization
# from keras.optimizers import Adam
# from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier     
from sklearn.ensemble import RandomForestRegressor

In [65]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
# 只需zone為7，8，9，12，13，14，17，18，19的資料
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/train_hire_stats.csv')  
df = df[(df.Zone_ID==7) | (df.Zone_ID==8) | (df.Zone_ID==9) | (df.Zone_ID==12) | (df.Zone_ID==13) | (df.Zone_ID==14) | (df.Zone_ID==17) | (df.Zone_ID==18) | (df.Zone_ID==19)]

arr = ['2016-02-08', '2016-02-09', '2016-02-10', '2016-02-11', '2016-02-12', '2016-02-29', '2016-04-04', '2016-04-05', '2016-06-09', '2016-06-10', '2016-09-15', '2016-09-16', 
       '2016-10-10', '2017-01-02', '2017-01-27', '2017-01-30']

df['holiday'] = 0
def cal_holiday(row):
  for i in arr:
    if (row['Date'] == i):
      row['holiday'] = 1
  return row
df = df.apply(cal_holiday, axis=1) 

df['Date'] = pd.to_datetime(df['Date'])
df['month'] = df['Date'].dt.month
df['week']=df['Date'].dt.dayofweek
df.drop(["Date"], axis=1)
df = df[['Zone_ID','Hour_slot','month','week','holiday','Hire_count']]
df
# sns.pairplot(df)

,Zone_ID,Hour_slot,month,week,holiday,Hire_count
52704,7,0,2,0,0,0
52705,7,1,2,0,0,0
52706,7,2,2,0,0,0
52707,7,3,2,0,0,0
52708,7,4,2,0,0,0
...,...,...,...,...,...,...
166891,19,19,1,1,0,16
166892,19,20,1,1,0,24
166893,19,21,1,1,0,19
166894,19,22,1,1,0,12


In [67]:
X = df.drop('Hire_count', axis=1)
y = df['Hire_count']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=101)

# model = RandomForestRegressor(n_estimators=80,bootstrap=True,max_features='auto',oob_score =True, random_state = 35,min_samples_leaf=50)  #10.6
# model = RandomForestRegressor(n_estimators=100,bootstrap=True,max_features='auto')  # 13.38 
# model = RandomForestClassifier(n_estimators=100,bootstrap=True,max_features='auto') # 16.7

model = RandomForestRegressor(n_estimators=80,bootstrap=True,max_features='auto',oob_score =True, random_state = 35,min_samples_leaf=50)
model.fit(X_train,y_train)
print(model)

rf_predictions = model.predict(X_test)

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
print ("rmes："+str(rmse(rf_predictions,y_test)) )

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=50,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=80, n_jobs=None, oob_score=True,
                      random_state=35, verbose=0, warm_start=False)
rmes：7.654819812640118


In [0]:
# imp=model.feature_importances_

In [0]:
# 修改資料
df3 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/test_hire_stats0.csv') 

df3['holiday'] = 0 # 先透過這種方式新增好欄位做準備，下面執行效能會好很多
def cal_holiday(row):
  for i in arr:
    if (row['Date'] == '2017-02-28'):
      row['holiday'] = 1
  return row
df3 = df3.apply(cal_holiday, axis=1) 

df3['Date'] = pd.to_datetime(df3['Date'])
df3['month'] = df3['Date'].dt.month
df3['week'] = df3['Date'].dt.dayofweek
df3.drop(["Date"], axis=1)
df3 = df3[['Zone_ID','Hour_slot','month','week','holiday','Hire_count']]

In [70]:
X3 = df3.drop('Hire_count',axis=1)
predictions = model.predict(X3)
# np.savetxt("foo.csv", predictions, delimiter=",")

dfwrite = pd.read_csv('/content/drive/My Drive/Colab Notebooks/test_hire_stats0.csv') 
dfwrite['Hire_count'] = predictions.astype(np.int32)
print(dfwrite)
dfwrite.to_csv('test_hire_stats.csv',index=False)

      Test_ID  Zone_ID        Date  Hour_slot  Hire_count
0           0        7  2017-02-01          0           0
1           1        7  2017-02-01          1           0
2           2        7  2017-02-01          2           0
3           3        7  2017-02-01          3           0
4           4        7  2017-02-01          4           0
...       ...      ...         ...        ...         ...
6043     6043       19  2017-02-28         19          26
6044     6044       19  2017-02-28         20          25
6045     6045       19  2017-02-28         21          24
6046     6046       19  2017-02-28         22          21
6047     6047       19  2017-02-28         23          21

[6048 rows x 5 columns]
